# **Libraries We'll Be Using**

In [0]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

from keras import regularizers
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation
from keras.utils import np_utils


from keras.optimizers import RMSprop, Adam
from keras.metrics import categorical_accuracy


from keras.callbacks import LearningRateScheduler
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

# **Loading Preprocessed Dataset**

**Loading the Data**

In [0]:
# open our uploaded file
svhn_data = h5py.File('SVHN_Preprocessed_Multi.h5', 'r')

# load the training, testing and validation set
X_train = svhn_data['X_train'][:]
y_train = svhn_data['y_train'][:]
X_test = svhn_data['X_test'][:]
y_test = svhn_data['y_test'][:]
X_val = svhn_data['X_val'][:]
y_val = svhn_data['y_val'][:]

# close the file
svhn_data.close()


# check that our datasets are correct
print('Training X Shape: ', X_train.shape)
print('Training Y Shape: ', y_train.shape)
print('Testing X Shape: ', X_test.shape)
print('Testing Y Shape: ', y_test.shape)
print('Validation X Shape: ', X_val.shape)
print('Validation Y Shape: ', y_val.shape)

**Display Some Training Images**

In [0]:
def plot_images(images, nrows, ncols, cls_true, cls_pred=None):
    
    # plot n rows, m columns
    fig, axes = plt.subplots(nrows, ncols, figsize=(16, 2*nrows))
    
    # randomly select images
    rs = np.random.choice(images.shape[0], nrows*ncols)
    
    # For every axes object in the grid
    for i, ax in zip(rs, axes.flat): 
        
        # get the image numbers
        true_number = ''.join(str(x) for x in cls_true[i] if x != 10)
        
        if cls_pred is None:
            title = "True: {0}".format(true_number)
        else:
            pred_number = ''.join(str(x) for x in cls_pred[i] if x != 10)
            title = "True: {0}, Pred: {1}".format(true_number, pred_number) 
        
        
        # display images with true label
        ax.imshow(images[i,:,:,0], cmap='binary')
        ax.set_title(title)   
        ax.set_xticks([]); ax.set_yticks([])
        
        
# plot some images from the training set
plot_images(X_train, 2, 8, y_train)

# **Prepare Train/Test/Validation Labels for Multi-Digit CNN Model**

In [0]:
def create_output_labels(labels):

  # 11 possible classes
  # classes for digits 0-9 and a class for no digit 
  classes = 11
  
  # output arrays
  dig1 = np.ndarray(shape=(len(labels), classes))
  dig2 = np.ndarray(shape=(len(labels), classes))
  dig3 = np.ndarray(shape=(len(labels), classes))
  dig4 = np.ndarray(shape=(len(labels), classes))
  dig5 = np.ndarray(shape=(len(labels), classes))

  # create one hot encoded labels
  for index,label in enumerate(labels):

      dig1[index,:] = np_utils.to_categorical(label[0], classes)
      dig2[index,:] = np_utils.to_categorical(label[1], classes)
      dig3[index,:] = np_utils.to_categorical(label[2], classes)
      dig4[index,:] = np_utils.to_categorical(label[3], classes)
      dig5[index,:] = np_utils.to_categorical(label[4], classes)

  return [dig1, dig2, dig3, dig4, dig5]
  

  
# create the train/test/val output arrays
train_labels = create_output_labels(y_train)
test_labels = create_output_labels(y_test)
val_labels = create_output_labels(y_val)


# check that the output shape is fine
print(np.shape(train_labels[0]))

# **Create a CNN Model**

**Multi-Output Model**

In [0]:
def cnn_model_multi():

  weight_decay = 1e-4
  classes = 11


  # LAYER 1
  model_input = Input(shape=X_train.shape[1:])

  M = Conv2D(32, (3,3), activation='elu', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(model_input)
  M = BatchNormalization()(M)
  M = Conv2D(32, (3,3), activation='elu', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(M)
  M = BatchNormalization()(M)
  M = MaxPooling2D(pool_size=(2,2))(M)
  M = Dropout(0.2)(M)


  # LAYER 2
  M = Conv2D(64, (3,3), activation='elu', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(M)
  M = BatchNormalization()(M)
  M = Conv2D(64, (3,3), activation='elu', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(M)
  M = BatchNormalization()(M)
  M = MaxPooling2D(pool_size=(2,2))(M)
  M = Dropout(0.3)(M)


  # LAYER 3
  M = Conv2D(128, (3,3), activation='elu', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(M)
  M = BatchNormalization()(M)
  M = Conv2D(128, (3,3), activation='elu', padding='same', kernel_regularizer=regularizers.l2(weight_decay))(M)
  M = BatchNormalization()(M)
  M = MaxPooling2D(pool_size=(2,2))(M)
  M = Dropout(0.4)(M)


  # OUTPUT
  Mout = Flatten()(M)

  dig1 = Dense(classes, activation='softmax', name = 'dig1')(Mout)
  dig2 = Dense(classes, activation='softmax', name = 'dig2')(Mout)
  dig3 = Dense(classes, activation='softmax', name = 'dig3')(Mout)
  dig4 = Dense(classes, activation='softmax', name = 'dig4')(Mout)
  dig5 = Dense(classes, activation='softmax', name = 'dig5')(Mout)

  out = [dig1, dig2, dig3, dig4, dig5]
  
  model = Model(input=model_input, output=out)
  return model

TEMP MODEL 2

In [0]:
def cnn_model_multi2():
  
  classes = 11
  model_input = Input(shape=X_train.shape[1:])
  
  
  # layer 1
  x = Conv2D(filters=48, kernel_size=5, padding="same")(model_input)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=2, strides=2, padding="same")(x)
  x = Dropout(0.25)(x)

  # layer 2
  x = Conv2D(filters=64, kernel_size=5, padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=2, strides=1, padding="same")(x)
  x = Dropout(0.25)(x)

  # layer 3
  x = Conv2D(filters=128, kernel_size=5, padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=2, strides=2, padding="same")(x)
  x = Dropout(0.25)(x)

  # layer 4
  x = Conv2D(filters=160, kernel_size=5, padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=2, strides=1, padding="same")(x)
  x = Dropout(0.25)(x)

  # layer 5
  x = Conv2D(filters=192, kernel_size=5, padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=2, strides=2, padding="same")(x)
  x = Dropout(0.25)(x)

  # layer 6
  x = Conv2D(filters=192, kernel_size=5, padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=2, strides=1, padding="same")(x)
  x = Dropout(0.25)(x)

  # layer 7
  x = Conv2D(filters=192, kernel_size=5, padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=2, strides=2, padding="same")(x)
  x = Dropout(0.25)(x)

  # layer 8
  x = Conv2D(filters=192, kernel_size=5, padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=2, strides=1, padding="same")(x)
  x = Dropout(0.25)(x)

  # layer 9
  x = Flatten()(x)
  x = Dense(units=3072)(x)
  x = Activation("relu")(x)
  x = Dropout(0.25)(x)

  # layer 10
  x = Dense(units=3072)(x)
  x = Activation("relu")(x)
  x = Dropout(0.25)(x)
  
  # output
  dig1 = Dense(classes, activation='softmax', name = 'dig1')(x)
  dig2 = Dense(classes, activation='softmax', name = 'dig2')(x)
  dig3 = Dense(classes, activation='softmax', name = 'dig3')(x)
  dig4 = Dense(classes, activation='softmax', name = 'dig4')(x)
  dig5 = Dense(classes, activation='softmax', name = 'dig5')(x)

  out = [dig1, dig2, dig3, dig4, dig5]
  model = Model(input=model_input, output=out)
  return model

# **Fit the Model and Display Accuracy**

**Compile, train, and save our CNN model**

In [0]:
# create and compile our model
model = cnn_model_multi()
model.compile(loss='categorical_crossentropy', 
              optimizer = RMSprop(lr=0.001, decay=1e-6), 
              metrics=["accuracy"])

# parameters for model fitting
batch_size = 128
epochs = 100

# train the model
model.fit(x=X_train, y=train_labels,
          validation_data=(X_val, val_labels),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

# save the model
model.save("SVHN_model_multi.h5")

In [0]:
def exp_decay(epoch):
    initial_lrate = 0.001
    k = 0.1
    lrate = initial_lrate * np.exp(-k*epoch)
    
    return lrate
   

opt = Adam(lr=0.001)
model = cnn_model_multi2()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
callbacks = [TensorBoard("logs", batch_size=128),
            LearningRateScheduler(exp_decay),
            EarlyStopping(monitor="val_loss", min_delta=0.05, patience=20)]


model.fit(x=X_train, y=train_labels,
          validation_data=(X_val, val_labels),
          batch_size=128,
          epochs=30,
          callbacks=callbacks,
          verbose=1)

model.save("SVHN_model_multi.h5")

**Check Model Accuracy**

In [0]:
# display final accuracy on validation set
scores = model.evaluate(X_val, val_labels, verbose=0)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))

# make our predictions on the testing set
test_predictions = model.predict(X_test)
accuracy = accuracy_score(test_labels, np.round(test_predictions))
print(accuracy)

**Display some of the predictions our model makes**

In [0]:
import random

# randomly select labels from the test dataset and see what the model predicts
for i in random.sample(range(0, len(test_labels[0])), 5):

  actual_labels = []
  predicted_labels = []

  for j in range(0,5):
      actual_labels.append(np.argmax(test_labels[j][i]))
      predicted_labels.append(np.argmax(test_predictions[j][i]))

  print("Actual labels:\t\t{}".format(actual_labels))
  print("Predicted labels:\t{}\n".format(predicted_labels))